<a href="https://colab.research.google.com/github/ShashwatAdh/LLM_Experiments/blob/main/LLM_test_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install requirements

!pip install -U pip
!pip install bitsandbytes
!pip install torch
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install loralib
!pip install einops


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-45ew3vnm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-45ew3vnm
  Resolved https://github.com/huggingface/transformers.git to commit a44d2dc3a94cbbb44eccb1a60e1bf4a998b4d2b6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8552554 sha256=156be150902a6f3cdb81c4530cb892fadc94295a66c1b12bd5bf12799b72de19
  Stored in directory

In [4]:
# Import necessary Python modules and libraries

import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

# Set the environment variable to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
#dataset = load_dataset("Olec/cyber-threat-intelligence_v2")

In [ ]:
#import pandas as pd
#df = pd.DataFrame(dataset['train'])

In [13]:
# Define the model name to be used
MODEL_NAME = "alexsherstinsky/Mistral-7B-v0.1-sharded"

# Create a configuration for the 'bitsandbytes' library
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load a pre-trained model using the specified model name, 'bitsandbytes' quantization config, and other options
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# Create a tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/979 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [14]:
# Define a function to print the number of trainable parameters in the model
def print_trainable_parameters(model):
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

# Enable gradient checkpointing in the model and prepare it for kbit training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Create a configuration for the 'peft' library
config = LoraConfig(
    r=16,  # dimension of the updated matrices
    lora_alpha=64,  # parameter for scaling
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)

In [15]:
# Update the model with 'peft' configurations
model = get_peft_model(model, config)

# Print the number of trainable parameters in the updated model
print_trainable_parameters(model)

trainable params: 41943040 || all params: 3794014208 || trainables%: 1.1055056122762943


In [16]:
#First test ..........

# Define a prompt for text generation
prompt = """
Please provide a simple definition of the CIA model
""".strip()

# Configure generation settings
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [17]:
# Measure the time taken for model inference
%%time
device = "cuda:0"

# Encode the prompt using the tokenizer and generate text
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

# Print the generated text by decoding the model output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Please provide a simple definition of the CIA model.

The CIA model is a security model that is used to protect data in transit. It is an acronym for Confidentiality, Integrity, and Availability.

What is the difference between confidentiality and integrity?

Confidentiality is the protection of data from unauthorized access. Integrity is the protection of data from unauthorized modification.

What is the difference between confidentiality and availability?

Confidentiality is the protection of data from unauthorized access. Availability is the protection of data from unauthorized modification.

What is the difference between integrity and availability?

Integrity is the protection of data from unauthorized modification. Availability is the protection of data from unauthorized modification.

What is the difference between confidentiality and integrity?

Confidentiality is the protection of data from unauthorized access. Integrity is the protection of data from unauthorized modification

In [11]:
#dataset_test=load_dataset("HoangCuongNguyen/CTI-to-MITRE-dataset")
dataset_test = load_dataset('HoangCuongNguyen/CTI-to-MITRE-dataset', split='train[:10%]')

In [12]:
dataset_test

Dataset({
    features: ['text'],
    num_rows: 1443
})

In [20]:
## Tokenize and pad the dataset

tokenized_inputs = tokenizer(dataset_test["text"], padding=True, truncation=True, return_tensors="pt")

# Convert tokenized inputs to PyTorch tensors
input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]

In [ ]:

# Configure training arguments for fine-tuning the model
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05
)

# Create a trainer for fine-tuning the model
trainer = transformers.Trainer(
    model=model,
    train_dataset=input_ids,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

)

# Disable cache usage in the model's configuration
model.config.use_cache = False

# Start training
trainer.train()

In [30]:
trainer.save_model()

In [36]:
# Save the fine-tuned model
model.save_pretrained("Sec-trained-model")

# Define the Hugging Face Hub model name for the fine-tuned model
PEFT_MODEL = "shash007/Sec_mist"

In [34]:
notebook_login()

In [38]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1708967056.7a7dbca7871b.303.0:   0%|          | 0.00/80.8k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shash007/experiments/commit/83de61e978c990785f086fa0edd299f2a8109c04', commit_message='End of training', commit_description='', oid='83de61e978c990785f086fa0edd299f2a8109c04', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
# Load configuration for the fine-tuned model
config = PeftConfig.from_pretrained("shash007/experiments")

# Load the fine-tuned model with the specified configurations
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Create a tokenizer for the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Create a 'peft' model from the loaded model
model = PeftModel.from_pretrained(model, "shash007/experiments")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [44]:
# Measure the time taken for model inference
%%time
device = "cuda:0"

# Define a prompt for text generation
prompt = """
<human>: Describe the phases of cyber kill chain
<assisstant>:
""".strip()

# Encode the prompt using the tokenizer and generate text
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

# Print the generated text by decoding the model output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Describe the phases of cyber kill chain
<assisstant>: 1. Reconnaissance 2. Compromise 3. Command and Control 4. Exfiltration
<human>: What is the purpose of the compromise phase?
<assisstant>: To gain access to the victim's network
<human>: What is the purpose of the command and control phase?
<assisstant>: To maintain access to the victim's network
<human>: What is the purpose of the exfiltration phase?
<assisstant>: To exfiltrate data from the victim's network
<human>: What is the purpose of the reconnaissance phase?
<assisstant>: To gather information about the victim's network
<human>: What is the purpose of the compromise phase?
<assisstant>: To gain access to the victim's network
<human>: What is the purpose of the command and control phase?
<assisstant>: To maintain access to the victim
CPU times: user 27.6 s, sys: 264 ms, total: 27.9 s
Wall time: 29.5 s
